## Imports

In [0]:
import sys
import os
sys.path.append(os.path.abspath('../utils/'))

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType
import json
from dwh_utils import *
from transformation_utils import *

## Assumption
- `spark.read` is a generic way of reading data files for this project. 
- In production, it would be reading just new/updated files

In [0]:
df = spark.read.option("multiLine", True).json("/Volumes/acme_ecommerce/source/data-sources/orders/Orders.json").withColumn("ingest_timestamp", F.current_timestamp()).select("*", "_metadata.file_path")

## Transformations

In [0]:
df = normalize_column_names(spark=spark, df=df)

## Saving to managed delta table

In [0]:
write_to_delta_with_cdc_by_name(
    spark=spark, df=df, catalog="acme_ecommerce", schema="bronze", table="orders", merge_keys=["order_id", "row_id"], append_only=True
)